**Importing Libraries**

importing traing data and test data and drop Id

In [ ]:
import numpy as np
import pandas as pd
import math
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import Ridge
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV ,RepeatedKFold
from sklearn.model_selection import train_test_split,cross_val_score, KFold 
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.metrics import classification_report, confusion_matrix
import sklearn.metrics as sklm
import matplotlib.pyplot as plt

In [ ]:
data_train = pd.read_csv('../input/house-prices-advanced-regression-techniques/train.csv')
data_test = pd.read_csv('../input/house-prices-advanced-regression-techniques/test.csv')
train= data_train.copy()
test = data_test.copy()
train = train.drop('Id', axis=1)
test = test.drop('Id', axis=1)

Exploring and cleanig the data

In [ ]:
train.head(5)



In [ ]:
train.info()

In [ ]:
test.info()

the "info"  function shows that there is a lot of missing data so as we calculate the percentage of the NaN value we found that **Alley** **PoolQC** **Fence** **MiscFeature** have more than 80% of data is NaN so the feature is not reliable we gonna exclude them

In [ ]:
# columns int tain set have more than 80% of nan 
nan_values_train = train.isnull().sum()
pourcentage_train =pd.DataFrame((np.array(nan_values_train)/1459)*100 ,index=nan_values_train.index)
pourcentage_train[pourcentage_train[0]>80]

In [ ]:
# columns int test set have more than 80% of nan values

nan_values_test = test.isnull().sum()
pourcentage_test = pd.DataFrame((np.array(nan_values_train)/1459)*100 ,index=nan_values_train.index)
pourcentage_test[pourcentage_test[0]>80]

In [ ]:
train = train.drop(['Alley','PoolQC','Fence','MiscFeature'], axis=1)

test = test.drop(['Alley','PoolQC','Fence','MiscFeature'], axis=1)


for other columns, we will change by the most **frequent** if the type of data **object** and with the **mean** if the type of data does **not object (int float...)**

In [ ]:
# filling the nan for tain set
missing_value_train = train.isnull().sum()

for col in list(missing_value_train.index):
    if train[col].dtype == 'object':
        train[col].fillna(train[col].value_counts().index[0], inplace=True)
    else:
        train[col].fillna(train[col].mean(), inplace=True)
        
# filling the nan for test set        
missing_value_test = test.isnull().sum()

for col in list(missing_value_test.index):
    if test[col].dtype == 'object':
        test[col].fillna(test[col].value_counts().index[0], inplace=True)
    else:
        test[col].fillna(test[col].mean(), inplace=True)


In [ ]:
# Encoding categorical train data

train=train.apply(LabelEncoder().fit_transform)

In [ ]:
test=test.apply(LabelEncoder().fit_transform)

In [ ]:
print("Train: \n{}".format(train.dtypes.value_counts()))


In [ ]:
#split the train data to feautures X and target y
y = np.ravel(np.array(train[['SalePrice']]))            
X = train.drop(['SalePrice'], axis=1)
             
print(y.shape)
print(X.shape)


In [ ]:
 #Splitting the dataset into the Training set and Test set

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)


In [ ]:
# Feature Scaling

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:

#createing Root Mean Squared Logarithmic Error (RMSLE) function

def rmsle (y_test, y_pred):
    return round(np.sqrt(sklm.mean_squared_error(y_test, y_pred)),10)

In [ ]:
#set the KFold function
kfolds = KFold(n_splits=10, shuffle=True, random_state=42)


**> Finging the alpha parameter fo Ridge **

In [ ]:

# define model
model = Ridge()
# define model evaluation method
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# define grid
grid = dict()
grid['alpha'] = np.arange(0, 1, 0.01)
# define search
search = GridSearchCV(model, grid, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
# perform the search
results = search.fit(X_train, y_train)
# summarize
print('MAE: %.3f' % results.best_score_)
print('Config: %s' % results.best_params_)

In [ ]:

## set Ridge Regulariztion (L2)
## Find Alpha

ridge = Ridge()
param = {'alpha': [a for a in range(50, 70)]}

ridge_reg = GridSearchCV(ridge, param_grid=param, scoring='neg_mean_squared_error'
                     , cv=10)

ridge_reg.fit(X_train, y_train)
print(f"The best value in Alpha: {ridge_reg.best_params_}")
print(f"The best score: {math.sqrt(-ridge_reg.best_score_)}")

In [ ]:
ridge_mod = Ridge(alpha=58)
ridge_mod.fit(X_train, y_train)
ridge_mod_train = ridge_mod.predict(X_train)
ridge_mod_test = ridge_mod.predict(X_test)

print(f'Root Mean Square Error train =  {rmsle(y_train, ridge_mod_train)}')
print(f'Root Mean Square Error test =  {rmsle(y_test, ridge_mod_test)}')   

MSEs = cross_val_score(ridge_mod, X, y, 
                       scoring='neg_mean_squared_error', 
                       cv=kfolds)

for i,j in enumerate(MSEs):
    j= math.sqrt(np.mean(-j))
    print(f'Fold {i}: {round(j,4)}')
    
print(f'Mean RMSE in Ridge: {round(math.sqrt(np.mean(-MSEs)),10)}')

In [ ]:
from sklearn.linear_model import Lasso

lasso = Lasso()
params = {'alpha': [0.1, 1, 10]}

lasso_reg = GridSearchCV(lasso, param_grid=param, cv=kfolds, scoring='neg_mean_squared_error')

lasso_reg.fit(X_train, y_train)
print(f'The best value of lasso: {lasso_reg.best_params_}')

In [ ]:

lasso_mod = Lasso(alpha=50)
lasso_mod.fit(X_train, y_train)
lasso_mod_train = lasso_mod.predict(X_train)
lasso_mod_test = lasso_mod.predict(X_test)

print(f'Root Mean Square Error train =  {str(rmsle(y_train, lasso_mod_train))}')
print(f'Root Mean Square Error test =  {rmsle(y_test, lasso_mod_test)}')

Lasso_CV = Lasso(alpha=1)
MSEs = cross_val_score(lasso_mod, X, y, scoring='neg_mean_squared_error', cv=kfolds)

for i,j in enumerate(MSEs):
    j= math.sqrt(np.mean(-j))
    print(f'Fold {i}: {round(j,4)}')

print(f'Mean Lasso: {round(math.sqrt(np.mean(-MSEs)),10)}')

In [ ]:
## Set RandomForest model
from sklearn.ensemble import RandomForestRegressor
random_forest = RandomForestRegressor(n_estimators=1400,
                                      max_depth=13,
                                      min_samples_split=5,
                                      min_samples_leaf=5,
                                      max_features=None,
                                      random_state=42,
                                      oob_score=True
                                     )

random_for = random_forest.fit(X_train, y_train)
random_for_mod = random_for.predict(X_test)

print(f'Root Mean Square Error test = {rmsle(y_test, random_for_mod)}')

In [ ]:
from sklearn.ensemble import VotingRegressor

## Voting in esemble

vote_mod = VotingRegressor([('Ridge', ridge_mod), ('Lasso', lasso_mod), 
                             ('Random_forest', random_forest)])
vote= vote_mod.fit(X_train, y_train.ravel())
vote_pred=vote.predict(X_test)

print(f'Root Mean Square Error test = {rmsle(y_test, vote_pred)}')

In [ ]:
## Set XGBRegressor model
from xgboost import XGBRegressor
xgb_regress = XGBRegressor(learning_rate=0.01,
                         n_estimators=3600,
                         max_depth=4, min_child_weight=1,
                         gamma=0.6, scale_pos_weight=1, 
                         seed=27, reg_alpha=0.00006 )

xg_mod = xgb_regress.fit(X_train, y_train)
xg_pred = xg_mod.predict(X_test)

print(f'Root Mean Square Error test =  {rmsle(y_test, xg_pred)}') 

In [ ]:
from mlxtend.regressor import StackingCVRegressor
model = [ridge_mod, lasso_mod, vote_mod]

## Stacking in ensemble

stack = StackingCVRegressor(regressors=model, 
                           meta_regressor=xgb_regress, 
                            use_features_in_secondary=True)

stack_mod=stack.fit(X_train, y_train.ravel())
stacking_pred=stack_mod.predict(X_test)

print(f'Root Mean Square Error test = {rmsle(y_test, stacking_pred)}')

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
lr_list = [0.05, 0.075, 0.1, 0.25, 0.5, 0.75, 1]
for learning_rate in lr_list :
    gb_clf = GradientBoostingClassifier(n_estimators=10, learning_rate=learning_rate, max_features=2, max_depth=2, random_state=0)
    gb_clf.fit(X_train, y_train)
    stackin_pred = gb_clf.predict(X_train)
    print("Learning rate: ", learning_rate)
    print(f'Root Mean Square Error test = {rmsle(y_test, stacking_pred)}')

In [ ]:
lasso_mod_test_data_pred=ridge_mod.predict(test)

In [ ]:

submission = pd.DataFrame({
    "Id": data_test["Id"],
    "SalePrice": lasso_mod_test_data_pred
    })

In [ ]:
submission

In [ ]:
submission.to_csv("submission_test_set.csv", index=False)
submission.head()

In [ ]:
#comapre between pridcted result and data result
sale_price_compare = pd.DataFrame({
    "train sale price desc": train.describe().iloc[:, -1],
    "test sale price desc": submission.describe().iloc[:, -1]
    },index=['count','mean','std','min','25%','50%','75%','max' ])
sale_price_compare